In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import great_expectations as gx

In [3]:
context = gx.get_context(project_root_dir="../services", mode="file")

# Create a datastore

In [4]:
ds = context.sources.add_or_update_pandas(name="transformed_sample")
ds

PandasDatasource(type='pandas', name='transformed_sample', id=None, assets=[])

## Load data

In [5]:
from src.data import extract_data, preprocess_data
from src.utils import init_hydra

cfg = init_hydra('main')
df, _ = extract_data("v1.0", cfg)
X, y = preprocess_data(cfg, df)


/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/passlib/utils/__init__.py:854: DeprecationWarning: 'crypt' is deprecated and slated for removal in Python 3.13
  from crypt import crypt as _crypt



In [6]:
X.columns

Index(['Airline', 'Origin', 'Dest', 'CRSDepTime', 'CRSElapsedTime', 'Distance',
       'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'IATA_Code_Marketing_Airline', 'Operating_Airline',
       'IATA_Code_Operating_Airline', 'OriginStateFips', 'OriginWac',
       'DestStateFips', 'DestWac', 'CRSArrTime', 'Quarter_sin', 'Quarter_cos',
       'Month_sin', 'Month_cos', 'DayofMonth_sin', 'DayofMonth_cos',
       'DayOfWeek_sin', 'DayOfWeek_cos', 'CRSArrTime_HH', 'CRSArrTime_MM',
       'CRSArrTime_HH_sin', 'CRSArrTime_HH_cos', 'CRSArrTime_MM_sin',
       'CRSArrTime_MM_cos', 'CRSDepTime_HH', 'CRSDepTime_MM',
       'CRSDepTime_HH_sin', 'CRSDepTime_HH_cos', 'CRSDepTime_MM_sin',
       'CRSDepTime_MM_cos', 'CRSElapsedTime_HH', 'CRSElapsedTime_MM',
       'CRSElapsedTime_HH_sin', 'CRSElapsedTime_HH_cos',
       'CRSElapsedTime_MM_sin', 'CRSElapsedTime_MM_cos'],
      dtype='object')

## Add dataframe assets

In [7]:
ds_X = ds.add_dataframe_asset(name="X", dataframe=X)
ds_y = ds.add_dataframe_asset(name="y", dataframe=y)
ds

Saving 2 Fluent Datasources to /home/sv-cheats-1/Documents/PROJECTS/MLops-project/notebooks/../services/gx/great_expectations.yml
CSVAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution
DataFrameAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution
Saving 2 Fluent Datasources to /home/sv-cheats-1/Documents/PROJECTS/MLops-project/notebooks/../services/gx/great_expectations.yml
CSVAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution
DataFrameAsset.dict() - missing config_provider, skipping config substitution
DataFrameAsset.dict() - missing config_provider, skipping config substitution
PandasDatasource.dict() - missing config_provider, skipping config substitution


PandasDatasource(type='pandas', name='transformed_sample', id=None, assets=[DataFrameAsset(name='X', type='dataframe', id=None, order_by=[], batch_metadata={}), DataFrameAsset(name='y', type='dataframe', id=None, order_by=[], batch_metadata={})])

# Create a suite


In [8]:
context.add_or_update_expectation_suite("feature_transform")

{
  "expectation_suite_name": "feature_transform",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.18"
  }
}

# Define the validator

In [9]:
validator = context.get_validator(
    batch_request=ds_X.build_batch_request(X),
    expectation_suite_name="feature_transform",
)
validator.columns()

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

['Airline',
 'Origin',
 'Dest',
 'CRSDepTime',
 'CRSElapsedTime',
 'Distance',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'IATA_Code_Marketing_Airline',
 'Operating_Airline',
 'IATA_Code_Operating_Airline',
 'OriginStateFips',
 'OriginWac',
 'DestStateFips',
 'DestWac',
 'CRSArrTime',
 'Quarter_sin',
 'Quarter_cos',
 'Month_sin',
 'Month_cos',
 'DayofMonth_sin',
 'DayofMonth_cos',
 'DayOfWeek_sin',
 'DayOfWeek_cos',
 'CRSArrTime_HH',
 'CRSArrTime_MM',
 'CRSArrTime_HH_sin',
 'CRSArrTime_HH_cos',
 'CRSArrTime_MM_sin',
 'CRSArrTime_MM_cos',
 'CRSDepTime_HH',
 'CRSDepTime_MM',
 'CRSDepTime_HH_sin',
 'CRSDepTime_HH_cos',
 'CRSDepTime_MM_sin',
 'CRSDepTime_MM_cos',
 'CRSElapsedTime_HH',
 'CRSElapsedTime_MM',
 'CRSElapsedTime_HH_sin',
 'CRSElapsedTime_HH_cos',
 'CRSElapsedTime_MM_sin',
 'CRSElapsedTime_MM_cos']

### 1. Check that no null values exist

In [10]:
for column in validator.columns():
    validator.expect_column_values_to_not_be_null(column=column)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

### 2. Check that columns that were hashed are hashed properly

In [11]:
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose


for column in X.columns[X.dtypes == 'object']:
    validator.expect_column_values_to_be_between(
        column=column,
        min_value=0,
        max_value=999,
)
    
# Adds no expectations, since it is not known what columns were hashed

### 3. Check that datatypes are correct (everything is a number) 

In [12]:
for column in validator.columns():
    validator.expect_column_values_to_be_of_type(column=column, type_=str(X[column].dtype))

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

### 4. Check that cyclic data in encoded correctly

In [13]:
for column in filter(lambda c: c.endswith('cos') or c.endswith('sin'), validator.columns()):
    validator.expect_column_values_to_be_between(
        column=column,
        min_value=-1,
        max_value=1,
)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

/home/sv-cheats-1/Documents/PROJECTS/MLops-project/.venv/lib/python3.11/site-packages/great_expectations/expectations/expectation.py:1518: UserWarning: result_format configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

In [14]:
print("Total number of expectations:", len(validator.expectation_suite.expectations))

Total number of expectations: 112


In [15]:
# Save expectations and checkpoint
validator.save_expectation_suite(discard_failed_expectations=False)
checkpoint = context.add_or_update_checkpoint(
    name="transformations_checkpoint",
    validator=validator,
)

	112 expectation(s) included in expectation_suite.


In [16]:
checkpoint_result = checkpoint.run()

# Open report in browser
context.view_validation_result(checkpoint_result)

if checkpoint_result.success:
    print("Data quality verification passed successfully")
else:
    print("Data quality verification failed")

	112 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/333 [00:00<?, ?it/s]

Data quality verification passed successfully
